In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [21]:
#Tools Conv3D, MaxPool3D, relu, flatten, softmax
# in_channels = 3, 128x128
#will reside from 3, x, 640x480 in dataloader
# 3, 60, 224, 224
#400 total probabilities? -> even number

class Net(nn.Module):
    def __init__(self, num_words=340, frames=60,size=256):
        super().__init__()
        self.conv1 = nn.Conv3d(3, 32, 3, padding = 'same')
        self.conv2 = nn.Conv3d(32, 32, 3, padding = 'same')
        self.conv3 = nn.Conv3d(32, 32, 3, padding = 'same')
        self.conv4 = nn.Conv3d(32, 32, 3, padding = 'same')
        self.conv5 = nn.Conv3d(32, 64, 3, padding = 'same') 

        self.conv6 = nn.Conv3d(64, 64, 3, padding = 'same')
        self.conv7 = nn.Conv3d(64, 64, 3, padding = 'same')

        self.conv8 = nn.Conv3d(64, 128, 3, padding = 'same')
        self.conv9 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv10 = nn.Conv3d(128, 128, 3, padding = 'same')

        self.conv11 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv12 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv13 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv14 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv15 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv16 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv17 = nn.Conv3d(128, 256, 3, padding = 'same')

        self.pool1 = nn.MaxPool3d((1, 2, 2))
        self.pool2 = nn.MaxPool3d(2)
        self.relu = nn.ReLU()
        self.preds1 = nn.Linear(int(128*(frames/4)*(size/16)*(size/16)), 4096)
        self.preds2 = nn.Linear(4096, num_words)

    def forward(self, x):
        #C T H W = 3 self.frames self.size self.size
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        tmp = x
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = tmp + x
        x = self.pool1(F.relu(self.conv5(x))) # 64 self.frames self.size/2 self.size/2
        tmp = x
        x = self.relu(self.conv6(x))
        x = self.relu(self.conv7(x))
        x = tmp + x 
        x = self.relu(self.conv8(x))
        tmp = x 
        x = self.relu(self.conv9(x))
        x = self.relu(self.conv10(x)) 
        x = tmp + x 
        x = self.pool1(x) # C T H W = 128 self.frames self.size/4 self.size/4
        x = self.relu(self.conv11(x))
        tmp = x
        x = self.relu(self.conv12(x))
        x = self.relu(self.conv13(x))
        x = tmp + x
        x = self.pool2(x) # 256 self.frames/2 self.size/8 self.size/8
        tmp = x
        x = self.relu(self.conv14(x)) 
        x = self.relu(self.conv15(x)) 
        x = tmp + x
        x = self.pool2(x)
        tmp = x
        x = self.relu(self.conv16(x)) 
        x = self.relu(self.conv17(x)) 
        x = tmp + x # 256 self.frames/4 self.size/16 self.size/16

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.preds1(x)
        x = self.preds2(x)
        return x



In [22]:
x = torch.rand(1, 3, 60, 256, 256)

In [24]:
model = Net()

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 8053063680 bytes.

In [ ]:
model(x).cpu()

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 503316480 bytes.

In [ ]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

NameError: name 'model' is not defined

In [ ]:
for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    for data in trainloader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

NameError: name 'trainloader' is not defined